In [1]:
# Install RAPIDS 0.15.0

import sys
!cp ../input/rapids/rapids.0.15.0 /opt/conda/envs/rapids.tar.gz
!cd /opt/conda/envs/ && tar -xzvf rapids.tar.gz > /dev/null
sys.path = ["/opt/conda/envs/rapids/lib/python3.7/site-packages"] + sys.path
sys.path = ["/opt/conda/envs/rapids/lib/python3.7"] + sys.path
sys.path = ["/opt/conda/envs/rapids/lib"] + sys.path 
!cp /opt/conda/envs/rapids/lib/libxgboost.so /opt/conda/lib/

In [2]:
import os, gc

import numpy as np
import pandas as pd
from scipy import signal

from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.metrics import f1_score, accuracy_score
from sklearn.model_selection import KFold, StratifiedKFold

import cuml
from cuml.ensemble import RandomForestRegressor, RandomForestClassifier
import cudf

print("CUML version:", cuml.__version__)

CUML version: 0.15.0


In [3]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        if col!='open_channels':
            col_type = df[col].dtypes
            if col_type in numerics:
                c_min = df[col].min()
                c_max = df[col].max()
                if str(col_type)[:3] == 'int':
                    if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                        df[col] = df[col].astype(np.int8)
                    elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                        df[col] = df[col].astype(np.int16)
                    elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                        df[col] = df[col].astype(np.int32)
                    elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                        df[col] = df[col].astype(np.int64)  
                else:
                    if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                        df[col] = df[col].astype(np.float16)
                    elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                        df[col] = df[col].astype(np.float32)
                    else:
                        df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [4]:
# signal processing features
def calc_gradients(s, n_grads=4):
    '''
    Calculate gradients for a pandas series. Returns the same number of samples
    '''
    grads = pd.DataFrame()
    
    g = s.values
    for i in range(n_grads):
        g = np.gradient(g)
        grads['grad_' + str(i+1)] = g
        
    return grads

def calc_low_pass(s, n_filts=5):
    '''
    Applies low pass filters to the signal. Left delayed and no delayed
    '''
    wns = np.logspace(-2, -0.3, n_filts)
    
    low_pass = pd.DataFrame()
    x = s.values
    for wn in wns:
        b, a = signal.butter(1, Wn=wn, btype='low')
        zi = signal.lfilter_zi(b, a)
        low_pass['lowpass_lf_' + str('%.4f' %wn)] = signal.lfilter(b, a, x, zi=zi*x[0])[0]
        low_pass['lowpass_ff_' + str('%.4f' %wn)] = signal.filtfilt(b, a, x)
        
    return low_pass

def calc_high_pass(s, n_filts=5):
    '''
    Applies high pass filters to the signal. Left delayed and no delayed
    '''
    wns = np.logspace(-2, -0.1, n_filts)
    
    high_pass = pd.DataFrame()
    x = s.values
    for wn in wns:
        b, a = signal.butter(1, Wn=wn, btype='high')
        zi = signal.lfilter_zi(b, a)
        high_pass['highpass_lf_' + str('%.4f' %wn)] = signal.lfilter(b, a, x, zi=zi*x[0])[0]
        high_pass['highpass_ff_' + str('%.4f' %wn)] = signal.filtfilt(b, a, x)
        
    return high_pass

def calc_ewm(s, windows=[10, 100]):
    '''
    Calculates exponential weighted functions
    '''
    ewm = pd.DataFrame()
    for w in windows:
        ewm['ewm_mean_' + str(w)] = s.ewm(span=w, min_periods=1).mean()
        ewm['ewm_std_' + str(w)] = s.ewm(span=w, min_periods=1).std()
        
    # add zeros when na values (std)
    ewm = ewm.fillna(value=0)
        
    return ewm

def calc_signal_features(s):
    '''
    All calculations together
    '''
    low_pass = calc_low_pass(s)
    high_pass = calc_high_pass(s)
    ewm = calc_ewm(s)
    
    return pd.concat([s, low_pass, high_pass, ewm], axis=1)

def signal_features(s, signal_size=500000):
    '''
    Divide the signal in bags of "signal_size".
    Normalize the data dividing it by 15.0
    '''
    # normalize
    s = s / 15.0
    
    ls = []
    for i in range(s.shape[0]//signal_size):
        sig = s[i*signal_size:(i+1)*signal_size].copy().reset_index(drop=True)
        sig_featured = calc_signal_features(sig)
        ls.append(sig_featured)
    
    ls = pd.concat(ls, axis=0)
    return ls[ls.columns[1:]]

In [5]:
# rolling and aggreagate batch features
def rolling_features(df):
    for window in [10, 100]:
        # rolling
        df['mean_t' + str(window)] = df.groupby(['group'])['signal'].transform(lambda x: x.rolling(window).mean())
        df['std_t' + str(window)] = df.groupby(['group'])['signal'].transform(lambda x: x.rolling(window).std())
        df['var_t' + str(window)] = df.groupby(['group'])['signal'].transform(lambda x: x.rolling(window).var())
        df['q25_t' + str(window)] = df.groupby(['group'])['signal'].transform(lambda x: x.rolling(window).quantile(0.25))
        df['q50_t' + str(window)] = df.groupby(['group'])['signal'].transform(lambda x: x.rolling(window).quantile(0.50))
        df['q75_t' + str(window)] = df.groupby(['group'])['signal'].transform(lambda x: x.rolling(window).quantile(0.75))
        df['min_t' + str(window)] = df.groupby(['group'])['signal'].transform(lambda x: x.rolling(window).min())
        df['max_t' + str(window)] = df.groupby(['group'])['signal'].transform(lambda x: x.rolling(window).max())
        min_max = (df['signal'] - df['min_t' + str(window)]) / (df['max_t' + str(window)] - df['min_t' + str(window)])
        df['norm_t' + str(window)] = min_max * (np.floor(df['max_t' + str(window)]) - np.ceil(df['min_t' + str(window)]))
    return df.fillna(0)

def shifted_features(df, num_shift=5):
    steps = np.arange(1, num_shift+1, dtype=np.int32)
    steps = np.append(steps, -steps)
    for step in steps:
        df['signal_shift_' + str(step)] = df['signal'].shift(step, fill_value=-2.73).astype( np.float32 )
    return df

def add_category(df):
    # treat 10 open channels group as another category
    df["category"] = 0
    if df.shape[0] > 2_000_000:
        # train segments with more then 9 open channels classes
        df.loc[2_000_000:2_500_000, 'category'] = 1
        df.loc[4_500_000:5_000_000, 'category'] = 1
    else:
        # test segments with more then 9 open channels classes (potentially)
        df.loc[500_000:600_000-1, "category"] = 1
        df.loc[700_000:800_000-1, "category"] = 1
    return df

def add_features(df):
    df = shifted_features(df)
    df = rolling_features(df)
    df = add_category(df)
    df['signal_2'] = (df['signal'] ** 2)
    df = reduce_mem_usage(df).reset_index(drop=True)
    sg_df = signal_features(df['signal'])
    sg_df = reduce_mem_usage(sg_df).reset_index(drop=True)
    return pd.concat([df, sg_df], axis=1)

In [6]:
def load_data():
    train = pd.read_csv("../input/ion-clean/train_full_clean.csv")
    test = pd.read_csv("../input/ion-clean/test_full_clean.csv")
    sub = pd.read_csv("../input/liverpool-ion-switching/sample_submission.csv")
    train['signal'] = train['signal'].astype( np.float32 )
    train['open_channels'] = train['open_channels'].astype( np.float32 )
    test['signal'] = test['signal'].astype( np.float32 )
    return train, test, sub

def augment_data(df):
    aug_df = df[df["group"] == 5].copy()
    aug_df["category"] = 1
    aug_df["group"] = 10
    for col in ["signal", "open_channels"]:
        aug_df[col] += df[df["group"] == 8][col].values

    aug_df['category'] = aug_df['category'].astype( np.float32 )
    df = df.append(aug_df, sort=False)
    return df

def drop_columns(df, columns=('open_channels', 'time', 'group')):
    return df[[c for c in df.columns if c not in columns]]

In [7]:
train, test, sub = load_data()
train["group"] = np.arange(train.shape[0]) // 500_000
test["group"] = np.arange(test.shape[0]) // 100_000

train = add_features(train)
test = drop_columns(add_features(test)).values.astype( np.float32 )
train = augment_data(train)

oof_preds = np.zeros((len(train)))
pred_test = np.zeros((len(test)))

kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

for fold, (trn_ind, val_ind) in enumerate(kf.split(train, train["group"])):
    print(f'Fold {fold}')
    
    trn, val = train.iloc[trn_ind], train.iloc[val_ind]
    x_trn = drop_columns(trn).values.astype( np.float32 )
    x_val = drop_columns(val).values.astype( np.float32 )
    
    model = RandomForestClassifier( #RandomForestRegressor
            n_estimators=50,
            rows_sample = 0.4,
            max_depth=18,
            max_features=11,        
            split_algo=0,
            bootstrap=False, #Don't use repeated rows, this is important to set to False to improve accuracy
        ).fit( x_trn, trn.open_channels )
    
    pred_val = model.predict( x_val )
    oof_preds[val_ind] = pred_val  # np.round( pred_val )
        
    pred_test += model.predict( test ) / 5
    del model; _=gc.collect()

Mem. usage decreased to 324.25 Mb (54.1% reduction)
Mem. usage decreased to 267.03 Mb (72.0% reduction)
Mem. usage decreased to 122.07 Mb (55.6% reduction)
Mem. usage decreased to 106.81 Mb (72.0% reduction)
Fold 0
Fold 1
Fold 2
Fold 3
Fold 4


In [8]:
f1_score(train.open_channels, oof_preds, average="macro")

0.9426307262293996

In [9]:
sub.open_channels = np.round( pred_test ).astype(np.int32)
sub.to_csv("submission.csv", index=False, float_format='%.4f')